In [1]:
import pandas as pd
import re
import openai
import re
import requests
import pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity

API_KEY = "c1173c2b55054c53b18e9eed1db42bf6"
RESOURCE_ENDPOINT = "https://gpt-4-canadaeast.openai.azure.com/" 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
#Note: The openai-python library support for Azure OpenAI is in preview.
openai.api_version = "2023-05-15"



## 1. Data generation (Skip this step if data is already generated )

In [2]:
user_message = ""
def generate_titles():
    user_message =f""" 
    generate 100 titles of customer support articles in your industry, focusing on the areas of payroll and HR that support agent can look up to answer questions from customers who are employees with payroll managed by ADP.
    Output data into a list 
 
"""

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a customer support agent for ADP company"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            

titles = generate_titles()

In [3]:
titles = titles.split("\n")

In [4]:
titles[:10]

['1. Understanding Your Payslip: A Guide for ADP Payroll Users',
 '2. How to Access and Navigate ADP Employee Self-Service Portal',
 '3. Troubleshooting Common Payroll Issues for ADP Users',
 '4. Understanding Tax Deductions: A Guide for ADP Payroll Users',
 '5. How to Update Personal Information in ADP Employee Self-Service',
 '6. Navigating Time and Attendance Tracking in ADP for Employees',
 '7. ADP Mobile App: A User Guide for Employee Payroll Access',
 '8. Resolving Errors in ADP Payroll Processing: A Step-by-Step Guide',
 '9. Understanding Your Vacation and Leave Balances in ADP',
 '10. Employee Benefits Enrollment: A Guide for ADP Payroll Users']

In [5]:
import time
def generate_article(title):
    user_message =f""" 
    given this title of an article "{title}" that support agents can look up to answer questions from customers who are employees with payroll managed by ADP.
    Generate detail content of the article 
 
"""
    i=0
    while i<5:
        try:
            response = openai.ChatCompletion.create(
                engine="gpt-35-turbo", # engine = "deployment_name".
                messages=[
                    {"role": "system", "content": "You are a customer support agent for ADP company"},
                    {"role": "user", "content":user_message },
                ]
            )
            return response['choices'][0]['message']['content']
        except:
            i+=1
            time.sleep(5)
            

contents =[]
for title in titles:
    contents.append(generate_article(title))

In [6]:
import os
folder_path ="../../../data/agent_assistant"
os.makedirs(folder_path, exist_ok=True)
i=0
articles ={}
for title, content in zip(titles, contents):
    file_name = f"article_{i}"
    file_content = title+"\n\n"+content

    with open(os.path.join(folder_path,file_name), 'w') as file:
        file.write(file_content)

    articles[file_name]= file_content
    i+=1



#### Document Chunking 

In [7]:
MAX_SECTION_LENGTH = 2000
SENTENCE_SEARCH_LIMIT = 200
SECTION_OVERLAP = 200
verbose= True
def split_text(all_text):
    SENTENCE_ENDINGS = [".", "!", "?"]
    WORDS_BREAKS = [",", ";", ":", " ", "(", ")", "[", "]", "{", "}", "\t", "\n"]
    length = len(all_text)
    start = 0
    end = length
    chunk=0
    while start + SECTION_OVERLAP < length:
        last_word = -1
        end = start + MAX_SECTION_LENGTH

        if end > length:
            end = length
        else:
            # Try to find the end of the sentence
            while end < length and (end - start - MAX_SECTION_LENGTH) < SENTENCE_SEARCH_LIMIT and all_text[end] not in SENTENCE_ENDINGS:
                if all_text[end] in WORDS_BREAKS:
                    last_word = end
                end += 1
            if end < length and all_text[end] not in SENTENCE_ENDINGS and last_word > 0:
                end = last_word # Fall back to at least keeping a whole word
        if end < length:
            end += 1

        # Try to find the start of the sentence or at least a whole word boundary
        last_word = -1
        while start > 0 and start > end - MAX_SECTION_LENGTH - 2 * SENTENCE_SEARCH_LIMIT and all_text[start] not in SENTENCE_ENDINGS:
            if all_text[start] in WORDS_BREAKS:
                last_word = start
            start -= 1
        if all_text[start] not in SENTENCE_ENDINGS and last_word > 0:
            start = last_word
        if start > 0:
            start += 1

        section_text = all_text[start:end]
        chunk += 1
        yield (section_text, chunk)

        start = end - SECTION_OVERLAP
        
    if start + SECTION_OVERLAP < end:
        yield (all_text[start:end], chunk)


In [8]:
import json
import os
folder_path ="../../data/agent_assistant"
with open(os.path.join(folder_path, "articles.json"), "r") as file:
    articles = json.load(file)


In [9]:
openai.api_version = "2023-05-15"
import json
import time
chunked_articles=[]
for article_file,article_content in articles.items():
    article_chunks =[]
    for chunk_content, chunk_number in split_text(article_content):
        article_chunks.append(chunk_content)
    chunked_articles.append((article_file,article_content, article_chunks))
openai.api_version = "2022-12-01"
chunks_emb=[]
for article in chunked_articles:
    chunks = article[2]
    chunk_no =0
    for chunk in chunks:
        chunk_id= article[0]+"###"+ str(chunk_no)
        chunk_emb = get_embedding(chunk, engine = 'text-embedding-ada-002')
        chunks_emb.append((chunk_id, chunk, chunk_emb))
        chunk_no += 1
        

### Persist data

In [10]:
import json
folder_path ="../../data/agent_assistant"

with open(os.path.join(folder_path,"chunk_emb_map.json"), "w") as fp:
    json.dump(chunks_emb,fp) 



## 2. Assistant Design

### Load data

In [11]:
import json
import os
folder_path ="../../data/agent_assistant"
with open(os.path.join(folder_path, "chunk_emb_map.json"), "r") as file:
    chunks_emb = json.load(file)


### 1. Tool to generate conversation

In [12]:
openai.api_version = "2023-05-15"

def generate_conversation(streaming =False):
    user_message =f""" 
    Generate a phone conversation between an customer's employee and an ADP support agent in the area of HR and Payroll. The employee has question and the support agent tried to identity the problems and 
    and take time to use different search tool to come up with answer to the employee.
    Your response:
 
"""
    system_message = """
    You are a customer support agent for ADP company. 

    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ],
        stream=streaming
    )
    return response
            


### 2. Tool to extract problems from conversation. 

In [20]:
openai.api_version = "2023-05-15"

user_message = ""
def extract_problems(conversation):
    user_message =f""" 
    Follow this on going conversation below and extract problems that each party may need help with and formulate the search query to the knowledge base search tool.
    <<conversattion>>
    {conversation}
    <<conversattion>>
    Output your response in JSON format {{"problem":"summary of problem", "search_query":"content of search query"}}
    There can be more than 1 problem(s)
    Output just JSON, nothing else.
    Your response:
 
"""
    system_message = """
    You are a senior customer support agent for ADP company. You listen to the conversation between an agent and a customer and assist the agent to resolve the problem.
    You are given access to knowledge base search tool to find knowledge needed to find answer to questions. 

    """

    response = openai.ChatCompletion.create(
        timeout=100,
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            
conversation=generate_conversation()['choices'][0]['message']['content']
problems=extract_problems(conversation)
problems

'{"problem":"Employee is experiencing a discrepancy in their paycheck amount", "search_query":"paycheck discrepancy resolution ADP"}'

### 3. Tool to find article given a problem

##### Brute Force method

In [13]:
import numpy as np  
openai.api_version = "2022-12-01"

def find_article(question, topk=5):  
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    input_vector = get_embedding(question, engine = 'text-embedding-ada-002')
    best_file_name = None  
      
    # Compute cosine similarity between input vector and each label vector
    cosine_list=[]  
    for chunk_id,chunk_content, vector in chunks_emb:  
        #by default, we use embedding for the entire content of the topic (plus topic descrition).
        # If you you want to use embedding on just topic name and description use this code cosine_sim = cosine_similarity(input_vector, vector[0])
        cosine_sim = cosine_similarity(input_vector, vector) 
        cosine_list.append((chunk_id,chunk_content,cosine_sim ))
    cosine_list.sort(key=lambda x:x[2],reverse=True)
    cosine_list= cosine_list[:topk]
    print(cosine_list)
    best_chunks =[chunk[0] for chunk in cosine_list]
    contents = [chunk[1] for chunk in cosine_list]
    return best_chunks, contents
question = "Why is my paycheck smaller than usual"
find_article(question)

[('article_0###1', '\n\nYear-to-Date Totals\n\nYour paycheck will include the year-to-date totals for your income, taxes, and contributions. This information will help you keep track of your earnings over the course of the year and prepare for tax season.\n\nIn conclusion, understanding your paycheck is important in managing your finances and planning for your future. If you have any questions about your paycheck or any other payroll-related issues, contact ADP customer support for assistance.', 0.8095147981219576), ('article_7###0', '8. Understanding Your ADP Pay Stub: What You Need to Know \n\nIntroduction \nAs an employee whose payroll is managed by ADP, understanding your pay stub is important. Your pay stub is a statement that shows you how much money you earned for a particular pay period, as well as deductions made from your earnings. In this article, we will help you understand how to read and understand your ADP pay stub.\n\n1. Gross Pay \nThis is the total amount of money you

(['article_0###1',
  'article_7###0',
  'article_0###0',
  'article_7###1',
  'article_35###1'],
 ['\n\nYear-to-Date Totals\n\nYour paycheck will include the year-to-date totals for your income, taxes, and contributions. This information will help you keep track of your earnings over the course of the year and prepare for tax season.\n\nIn conclusion, understanding your paycheck is important in managing your finances and planning for your future. If you have any questions about your paycheck or any other payroll-related issues, contact ADP customer support for assistance.',
  '8. Understanding Your ADP Pay Stub: What You Need to Know \n\nIntroduction \nAs an employee whose payroll is managed by ADP, understanding your pay stub is important. Your pay stub is a statement that shows you how much money you earned for a particular pay period, as well as deductions made from your earnings. In this article, we will help you understand how to read and understand your ADP pay stub.\n\n1. Gross 

#### Vector Searh Lib (Faiss) method

In [15]:
import numpy as np  
import faiss

openai.api_version = "2022-12-01"
#Get the array of embeddings for all articles
chunk_ids =[]
emb_vectors = []
for chunk_id,chunk_content, vector in chunks_emb:  
    #by default, we use embedding for the entire content of the topic (plus topic descrition).
    # If you you want to use embedding on just topic name and description use this code cosine_sim = cosine_similarity(input_vector, vector[0])
    chunk_ids.append(chunk_id)
    emb_vectors.append(vector)
emb_vectors = np.float32(emb_vectors)
faiss.normalize_L2(emb_vectors)

index = faiss.IndexFlatIP(len(vector)) 
index.add(emb_vectors)


def find_article_emb_vec(question, topk=3):  
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    input_vector = get_embedding(question, engine = 'text-embedding-ada-002')
    input_vector = np.float32([input_vector])
    faiss.normalize_L2(input_vector)
    d,i = index.search(input_vector, k=topk)
    best_chunks = [chunks_emb[idx] for idx in i[0]]
    print(d)
    output_contents=[chunk[1] for chunk in best_chunks]
    chunks =[chunk[0] for chunk in best_chunks]
    return chunks, output_contents
question = "Why is my paycheck  smaller than usual"
find_article_emb_vec(question,5)

[[0.8133553  0.8012111  0.8003706  0.78765255 0.7852054 ]]


(['article_0###1',
  'article_0###0',
  'article_7###0',
  'article_7###1',
  'article_35###1'],
 ['\n\nYear-to-Date Totals\n\nYour paycheck will include the year-to-date totals for your income, taxes, and contributions. This information will help you keep track of your earnings over the course of the year and prepare for tax season.\n\nIn conclusion, understanding your paycheck is important in managing your finances and planning for your future. If you have any questions about your paycheck or any other payroll-related issues, contact ADP customer support for assistance.',
  "1. Understanding Your Paycheck: A Guide for ADP Payroll Customers \n\n1. Understanding Your Paycheck: A Guide for ADP Payroll Customers\n\nAs an ADP Payroll customer, you may receive your paycheck via direct deposit or a physical check. It is important to understand the different sections of your paycheck and what they represent. This guide will help you understand your paycheck and answer any questions you may h

### 4. Tool to answer question from the given problem

In [16]:
openai.api_version = "2023-03-15-preview"

def answer_assist(problem, search_query):

    articles, contents = find_article(search_query,2)
    articles_contents=""
    for article, content in zip(articles, contents):
        articles_contents += f""" 
        article:{article}
        content: {content}
    """
    articles_contents = f"""
    <<knowledge articles>>
    {articles_contents}
    <<knowledge articles>>
    """
    user_message =f""" 
    problem:{problem}
    {articles_contents}
    Your response:
"""
    system_message = """
    You are a senior customer support agent for ADP company. You listen to the conversation between an agent and a customer and assist the agent to resolve the problem.
    Given the question or problem statement and the knowledge article you have, give the answer.
    Rely solely to the guidance from the article.If the knowlege article is not relavant to the question, say you don't know. Do not make up your answer. 
    Cite the name of the knowledge article as source for your answer.
    Format:
    Answer: your answer to the question
    Sources: [source1, source2]
    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content'], articles_contents

question = "When do I receive my W2 form?"
answer, articles_contents = answer_assist(question,question)
print(answer)
print("---------------content-----------------")

print(articles_contents)

[('article_3###0', "4. Understanding Your W-2: A Guide for ADP Payroll Customers \n\nIf you are an employee whose payroll is managed by ADP, you will receive a W-2 form at the end of each fiscal year. This form summarizes your earnings and taxes paid throughout the year, and is necessary for filing your tax returns.\n\nTo help you understand the information displayed on your W-2 form, we've created this guide:\n\n1. Box 1 - Wages, tips, and other compensation: This box includes all taxable income earned during the year, including bonuses, overtime pay, and vacation pay.\n\n2. Box 2 - Federal income tax withheld: This box shows the total amount of federal income tax withheld from your paycheck throughout the year. This amount is based on your W-4 form, which you fill out when you are hired.\n\n3. Box 3 - Social Security wages: This box shows the total amount of wages that are subject to Social Security taxes. This includes all taxable income up to the Social Security wage base, which ch

### 5. Put tools all together

In [17]:
def recommend(conversation):
    i=0
    while (i<5): #handle wrong format output
        problems=extract_problems(conversation)
        try:
            problems=json.loads(problems)
            print("problems", problems)
            for problem in problems:
                answer, articles_contents = answer_assist(problem['problem'],problem['search_query'])
                print(answer)
                print("---------------content-----------------")

                print(articles_contents)
            break

        except Exception as e:
            print(e)
            print("problem parsing json, problems string is ", problems)
            i+=1

        



### One time conversation 

In [21]:
conversation =generate_conversation()['choices'][0]['message']['content']
print(f"Conversation {conversation}")
recommend(conversation)        

    

Conversation Support Agent: Thank you for calling ADP support. My name is Sarah. How can I assist you today?

Employee: Hi, yes. I have a question about my payroll. I noticed that my paycheck amount seems to be incorrect.

Support Agent: I apologize for the inconvenience. I'll do my best to help you out. Could you please provide me with your employee ID so I can access your account?

Employee: Sure, my employee ID is 6789.

Support Agent: Thank you. Let me just pull up your information in our system. Please bear with me for a moment.

(Employee waits patiently)

Support Agent: Okay, I have your payroll records in front of me. Can you tell me which pay period you are referring to and what the discrepancy is?

Employee: It's for the last pay period. The amount seems to be lower than what it should be based on my hours worked and the agreed-upon rate.

Support Agent: I understand your concern. Let me investigate this further for you. While I'm doing that, may I ask if you made any changes

### Simulate a running conversation 

In [22]:
import concurrent
openai.api_version = "2023-05-15"
import time
conversation_pause_duration=2
agent_assist_freq=5
conversation_buffer =[]
def stream_conversation(conversation_buffer, pause_duration=5):
    responses = generate_conversation(True)
    conversation_counter =0
    old_conversation_counter =0
    for response in responses:
        content = response['choices'][0]["delta"].get("content","")
        conversation_buffer.append(content) 
        if "\n"  not in content:
            print(content, end="")
        else:
            conversation_counter+=1
            if conversation_counter > old_conversation_counter+2:
                conversation = "".join(conversation_buffer)
                print("starting recommendation")
                print("conversation: ", conversation)
                recommend(conversation)
                print("ending recommendation")
                old_conversation_counter=conversation_counter



        
stream_conversation(conversation_buffer,conversation_pause_duration)

Support Agent: Thank you for calling ADP support. My name is Sarah. How can I assist you todayEmployee: Hi Sarah, my name is John. I'm having trouble with my paycheck deductions. I noticed that I'm being deducted more than usual for taxes this month. Can you help me understand whySupport Agent: Of course, John. I'm sorry to hear about the issue you're facing. Let me look into your account to see what might be causing the increased tax deductions. Can you please provide me with your employee ID or any other identifying informationstarting recommendation
conversation:  Support Agent: Thank you for calling ADP support. My name is Sarah. How can I assist you today?

Employee: Hi Sarah, my name is John. I'm having trouble with my paycheck deductions. I noticed that I'm being deducted more than usual for taxes this month. Can you help me understand why?

Support Agent: Of course, John. I'm sorry to hear about the issue you're facing. Let me look into your account to see what might be causing

KeyboardInterrupt: 